In [1]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
import os
os.chdir('E:/thuc/doan2/data')  # Đổi thư mục làm việc

In [3]:
# Cấu hình đường dẫn
raw_dir = 'raw'  # Thư mục chứa các thư mục con của từng người
processed_dir = 'processed'# Thư mục lưu ảnh đã xử lý
pairs_dir = 'pairs'

In [4]:
# Khởi tạo MTCNN detector
detector = MTCNN()

In [5]:
# Hàm phát hiện và căn chỉnh khuôn mặt
def detect_and_align_face(image_path):
    """
    Phát hiện và căn chỉnh khuôn mặt trong ảnh.
    Trả về ảnh khuôn mặt đã căn chỉnh nếu phát hiện được, nếu không trả về None.
    """
    try:
        # Đọc ảnh
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Phát hiện khuôn mặt
        faces = detector.detect_faces(img_rgb)

        if len(faces) > 0:
            # Chọn khuôn mặt đầu tiên
            face = faces[0]
            x, y, w, h = face['box']
            # Cắt khuôn mặt
            face_aligned = img_rgb[y:y+h, x:x+w]
            return face_aligned
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {image_path}: {e}")
    return None


In [6]:
# Duyệt qua các thư mục con trong thư mục raw (từng người)
person_dirs = os.listdir(raw_dir)

for person_dir in tqdm(person_dirs, desc="Đang xử lý ảnh..."):
    person_dir_path = os.path.join(raw_dir, person_dir)
    
    if os.path.isdir(person_dir_path):
        # Tạo thư mục con trong processed cho mỗi người
        person_processed_dir = os.path.join(processed_dir, person_dir)
        if not os.path.exists(person_processed_dir):
            os.makedirs(person_processed_dir)
        
        # Đọc tất cả các file hình ảnh trong thư mục của người này
        file_list = os.listdir(person_dir_path)
        
        for file_name in file_list:
            file_path = os.path.join(person_dir_path, file_name)
            
            if file_name.endswith('.jpg') or file_name.endswith('.png'):
                # Phát hiện và căn chỉnh khuôn mặt
                aligned_face = detect_and_align_face(file_path)
                
                if aligned_face is not None:
                    # Lưu ảnh đã căn chỉnh vào thư mục processed của người đó
                    save_path = os.path.join(person_processed_dir, file_name)
                    aligned_face_bgr = cv2.cvtColor(aligned_face, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(save_path, aligned_face_bgr)

print("Xử lý ảnh hoàn tất. Dữ liệu đã được lưu vào thư mục 'processed'.")

Đang xử lý ảnh...:   0%|          | 0/31 [00:00<?, ?it/s]

1/1 [==============================] - 0s 25ms/step


Đang xử lý ảnh...:   3%|▎         | 1/31 [00:49<24:58, 49.96s/it]

1/1 [==============================] - 0s 18ms/step


Đang xử lý ảnh...:   3%|▎         | 1/31 [02:47<1:23:54, 167.83s/it]
c:\Users\HUY THONG\AppData\Local\Programs\Python\Python39\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\HUY THONG\AppData\Local\Programs\Python\Python39\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\HUY THONG\AppData\Local\Programs\Python\Python39\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\HUY THONG\AppData\Local\Programs\Python\Python39\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 

In [7]:
# Tạo cặp ảnh (positive và negative)
pairs = []
labels = []

processed_people = os.listdir(processed_dir)

# Tạo cặp positive (cùng một người)
for person in processed_people:
    person_path = os.path.join(processed_dir, person)
    images = os.listdir(person_path)
    if len(images) > 1:  # Chỉ tạo cặp nếu có ít nhất 2 ảnh
        for i in range(len(images) - 1):
            pairs.append([os.path.join(person_path, images[i]),
                          os.path.join(person_path, images[i + 1])])
            labels.append(1)  # Nhãn 1 là positive

# Tạo cặp negative (khác người)
for i in range(len(processed_people) - 1):
    person_1 = processed_people[i]
    person_2 = processed_people[i + 1]
    images_1 = os.listdir(os.path.join(processed_dir, person_1))
    images_2 = os.listdir(os.path.join(processed_dir, person_2))
    if images_1 and images_2:  # Đảm bảo thư mục không rỗng
        pairs.append([os.path.join(processed_dir, person_1, images_1[0]),
                      os.path.join(processed_dir, person_2, images_2[0])])
        labels.append(0)  # Nhãn 0 là negative

# Lưu cặp ảnh và nhãn
np.save(os.path.join(pairs_dir, "pairs.npy"), np.array(pairs))
np.save(os.path.join(pairs_dir, "labels.npy"), np.array(labels))

print(f"Cặp ảnh và nhãn đã được lưu tại: {pairs_dir}")

Cặp ảnh và nhãn đã được lưu tại: pairs
